# Imports and Plane variables

In [ ]:
import json
import pandas as pd
import datetime
import requests
from collections import defaultdict
import time

In [ ]:
# define your Plane variables : change this with your values
workspace_slug = "your_workspace_name"
domain = "your workspace domain : e.g sub.domain.com"
api_key = "plane_api_hexa"
headers = {"x-api-key": api_key}

In [ ]:
# table containing user names as per Jira (Assignee), and user ids as per Plane.
# in plane, user ids can be retrieved by created a dummy project, add all users as members, create tasks and assign one to each user,
# and use the API to see Assignees
# change this with your values
user_table = { "User 1 as per Jira Assigne column in exports" : "user_1_id_in_plane"}

In [ ]:
# Table containing Jira assignees/users ids and Jira assignes/users names : needed to parse comments in Jira
# put a default users for comments for unknown members ( e.g : former employee)
# change this with your values
jira_user_table = {
    "default" : "your default user name",
    "user_1_id_in_jira" : "user 1",
}

In [ ]:
# map priority levels in Jira versus Plane (keys are jira's, values are plane's)
# don't change it if this default mapping is fine
priority_table = {"Lowest" : "low", "Low" : "low", "Medium" : "medium", "High": "high", "Highest": "urgent" }

In [ ]:
# correspondance tables for states between jira & planes. Keys are jira states, values are Plane states label
# don't change it if this default mapping is fine
state_correspondance_table = {"Done" : "Done", "In Progress" : "In Progress", "To Do": "Todo", "Canceled" : "Cancelled", "Closed" : "Done"}

# Import and variables to manage attachments with GDrive

In [ ]:
# import here (via a file) your latest cookies for jira website from browser using an extension such as Cookie Editor
# it will be needed to access Jira attachments
# update the cookie_filename accordingly
cookie_filename = '/content/atlassian_cookie.json'

with open(cookie_filename) as f:
    cookie = json.load(f)

cookies_dict = {cookie['name']: cookie['value'] for cookie in cookie}

In [ ]:
# imports related to using GDrive as destination for attachment (needed until attachment upload is available and documented in Plane API)
from google.colab import drive
from googleapiclient.discovery import build
from io import BytesIO
from google.colab import auth

In [ ]:
# init : mount google drive to push images from Jira
# Mount Google Drive
drive.mount('/content/gdrive')
auth.authenticate_user()

# Replace with your actual folder path in Google Drive where you want to store attachments
# The folder should exist : create it before running the cell
# it can be a shared drive
# change this with your values
gdrive_folder_path = '/content/gdrive/Shareddrives/your_attachment_destination/'
%cd $gdrive_folder_path

# Import / create in Plane: functions (run as is)

In [ ]:
def send_request(method, url, headers,*arg):
    try:
        payload = arg[0]
        response = requests.request(method, url, headers=headers, json=payload)
        print(response)
        if response.status_code in [200, 201, 204]:
            response = json.loads(response.text)
            return response
        elif response.status_code == 429:
            print("Pausing for one minute")
            time.sleep(60)
            response = requests.request(method, url, headers=headers, json=payload)
            response = json.loads(response.text)
            return response
        else:
            print('Error : ', response, response.text)
    except:
        response = requests.request(method, url, headers=headers)
        print(response)
        if response.status_code in [200, 201, 204]:
            response = json.loads(response.text)
            return response
        elif response.status_code == 429:
            print("Pausing for one minute")
            time.sleep(60)
            response = requests.request(method, url, headers=headers)
            response = json.loads(response.text)
            return response
        else:
            print('Error : ', response, response.text)

In [ ]:
# function to handle module creation corresponding to Jira Epics
# assumes df contains only Epics
# NOT USED ANYMORE
def create_module(domain, workspace_slug, project_id, df, module_table):
    #create modules in Plane based on Jira Epics info. We use keys and not ids as EPIC are linked via keys in the jira export
    #some exports do not use Custom Epic links, only parents => need for a v2
    url = f"https://{domain}/api/v1/workspaces/{workspace_slug}/projects/{project_id}/modules/"
    for _, row in df.sort_values(by=["Issue id"]).iterrows():
        payload = {
            "name": row['Summary'],
            "status": 'in-progress',
            }
        description = desc_to_html(row['Description'])
        if description != "":
            payload['description_html'] = description
        if get_assignee(row['Assignee']) != "":
            payload["lead"] = [get_assignee(row['Assignee'])]
        response = send_request("POST", url, headers, payload)
        module_table[row['Issue key']]=response['id']

    return module_table


# function to handle module creation corresponding to Jira Epics
# assumes df contains only Epics
def create_module_v2(domain, workspace_slug, project_id, df, module_table):
    #create modules in Plane based on Jira Epics info. We use ids as EPIC will be linked to issues via parent ids
    url = f"https://{domain}/api/v1/workspaces/{workspace_slug}/projects/{project_id}/modules/"
    for _, row in df.sort_values(by=["Issue id"]).iterrows():
        payload = {
            "name": row['Summary'],
            "status": 'in-progress',
            }
        description = desc_to_html(row['Description'])
        if description != "":
            payload['description_html'] = description
        if get_assignee(row['Assignee']) != "":
            payload["lead"] = [get_assignee(row['Assignee'])]
        response = send_request("POST", url, headers, payload)
        module_table[row['Issue id']]=response['id']

    return module_table

In [ ]:
# function to handle cycle creation corresponding to Jira sprints

def create_cycle(domain, workspace_slug, project_id, sprint_table):
    #create cycles in Plane based on Jira sprints info
    # NOT USED ANYMORE

    url = f"https://{domain}/api/v1/workspaces/{workspace_slug}/projects/{project_id}/cycles/"

    cycle_table = {}
    for i in range(len(sprint_table)):
        payload = {"name": sprint_table[i]['name'],
                "start_date" : sprint_table[i]['startDate'].strftime('%Y-%m-%d'),
                "end_date": sprint_table[i]['endDate'].strftime('%Y-%m-%d')
                }
        response = send_request("POST", url, headers, payload)
        cycle_table[sprint_table[i]['name']]=response['id']

    return cycle_table

def create_cycle_no_end_date(domain, workspace_slug, project_id, cycle_table, sprint_table):
    #create cycles in Plane based on Jira sprints info
    #expect sprint_table to be a list of (only one) sprint
    #update cycle_table instead of creating one from scratch

    url = f"https://{domain}/api/v1/workspaces/{workspace_slug}/projects/{project_id}/cycles/"

    tomorrow = datetime.datetime.now() + datetime.timedelta(days=1)
    #cycle_table = {}
    for i in range(len(sprint_table)):
        payload = {"name": sprint_table[i]['name'],
                "start_date" : sprint_table[i]['startDate'].strftime('%Y-%m-%d'),
                "end_date" : tomorrow.strftime('%Y-%m-%d')
                }
        response = send_request("POST", url,headers,payload)
        print('Cycle creation:')
        print(response)
        cycle_table[sprint_table[i]['name']]=response['id']

    return cycle_table

def update_one_cycle_end_date(domain, workspace_slug, project_id, cycle_table, sprint_table):
    #update cycle end date
    #except sprint_table to be a list of only one sprint
    url = f"https://{domain}/api/v1/workspaces/{workspace_slug}/projects/{project_id}/cycles/{cycle_table[sprint_table[0]['name']]}/"
    payload = {
        "name": sprint_table[0]['name'],
        "start_date" : sprint_table[0]['startDate'].strftime('%Y-%m-%d'),
        "end_date" : sprint_table[0]['endDate'].strftime('%Y-%m-%d')
        }
    response = send_request("PATCH", url,headers, payload)
    print('Cycle update:')
    print(response)


def read_all_cycle(domain, workspace_slug, project_id):
    # not fully tested - not used
    url = f"https://{domain}/api/v1/workspaces/{workspace_slug}/projects/{project_id}/cycles/"

    cycle_table = {}

    response = send_request("GET", url, headers)

    #print(response)
    for i in range(len(response['results'])):
        cycle_table[response['results'][i]['name']]= response['results'][i]['id']
    return cycle_table


In [ ]:
# get state list, complete state correspondance dict
def complete_state_dict(domain, workspace_slug, project_id):
    url = f"https://{domain}/api/v1/workspaces/{workspace_slug}/projects/{project_id}/states/"

    response = send_request("GET", url, headers)

    state_dict = {}

    for items in response["results"]:
        state_dict[items['name']] = items['id']
    print(state_dict)

    for k,v in state_correspondance_table.items():
        project_jira_state_dict[k] = state_dict[v]
    #project_jira_state_dict['Done'] = state_dict['Done']
    #project_jira_state_dict['In Progress'] = state_dict['In Progress']
    #project_jira_state_dict['To Do'] = state_dict['Todo']
    #project_jira_state_dict['Canceled'] = state_dict['Cancelled']
    print(project_jira_state_dict)
    return project_jira_state_dict



In [ ]:
# get project Labels (create a Label Bug, in red, if needed)
def create_or_get_label_bug(domain, workspace_slug, project_id):
    label_bug = ""
    url = f"https://{domain}/api/v1/workspaces/{workspace_slug}/projects/{project_id}/labels/"

    response = send_request("GET", url, headers)

    try:
        for items in response['results']:
            if items['name'] == 'Bug' or items['name'] == 'bug':
                label_bug = items['id']
    except:
        pass

    if label_bug == "":
        payload = {"name": "Bug", "color": '#eb144c'}
        response = send_request("POST", url, headers, payload)
        label_bug = response['id']
    return label_bug


In [ ]:
def get_assignee(item):
    try:
        return user_table[item]
    except:
        return ""


def desc_to_html(description):
    html_desc= ""
    if type(description) is not str:
        return ""
    for lines in description.splitlines():
        html_desc += "<p>" + lines + "</p>"
    return html_desc

def build_comment_payload(comments):
    #print(comments)
    # first version : assumes only one comment is present. assumes comments does not include ';'
    split_comments = comments.split(';')
    #print(split_comments)
    # O should be timestamp, 1 user id in jira, 2 actual comment
    try:
        payload = {
            'comment_html' : desc_to_html(split_comments[2]),
            'actor' : user_table[jira_user_table[split_comments[1]]]
        }
    except KeyError:
        payload = {
            'comment_html' : desc_to_html(split_comments[2]),
            'actor' : user_table[jira_user_table['default']]
        }
    return payload

def find_max_sprint_nb_in_df(df):
    # not needed anymore
    columns = df.columns.tolist()
    sprint_column = []
    for column in columns:
        if column.startswith('Sprint'):
            sprint_column.append(column)
    return len(sprint_column)

def assess_sprint_nb(row, max_sprint):
    # not needed anymore
    for count in range(max_sprint-1, 0, -1):
        if type(row[f'Sprint.{count}']) is str and row[f'Sprint.{count}'] != '':
            return row[f'Sprint.{count}']

    if type(row['Sprint']) is str and row['Sprint']!= '':
        return row['Sprint']
    else:
        #print(row)
        return None

def add_lastsprint_df(df):
    columns = df.columns.tolist()
    sprint_column = []
    for column in columns:
        if column.startswith('Sprint'):
            sprint_column.append(column)
    df['Last Sprint'] = df[sprint_column].max(axis=1)
    return df


In [ ]:
# create issues corresponding to Jira issues, bugs,tasks or sub-tasks and assign them to the proper cycle
def create_issues(domain, workspace_slug, project_id, df, project_jira_state_dict, label_bug, issue_id_table):

    url = f"https://{domain}/api/v1/workspaces/{workspace_slug}/projects/{project_id}/issues/"

    # create issues without sprint, comments and parent relationships

    for _, row in df.sort_values(by=["Issue id"]).iterrows():
        payload = {
            "name": row['Summary'],
            "state": project_jira_state_dict[row['Status']],
            "priority" : priority_table[row['Priority']]
            }
        description = desc_to_html(row['Description'])
        if description != "":
            payload['description_html'] = description
        if get_assignee(row['Assignee']) != "":
            payload["assignees"] = [get_assignee(row['Assignee'])]
        if row['Issue Type'] == 'Bug':
            payload['labels'] = [label_bug]
        #print(payload)
        response = send_request("POST", url, headers, payload )
        issue_id_table[row['Issue id']]= response['id']
    return issue_id_table

def add_comments(domain, wokspace_slug, project_id, df, issue_id_table):
    # add comments. Comments may not exist in the df, so check first
    if 'Comment' not in df.columns:
        return None
    count = 0
    for columns in df.columns:
        if columns.startswith('Comment'):
            count += 1
    for _, row in df.sort_values(by=["Issue id"]).iterrows():
        if type(row['Comment']) is str and row['Comment'] != '':
            url = f"https://{domain}/api/v1/workspaces/{workspace_slug}/projects/{project_id}/issues/{issue_id_table[row['Issue id']]}/comments/"
            payload = build_comment_payload(row['Comment'])
            #print(payload)
            response = send_request("POST", url, headers, payload)
        for i in range(1, count):
            if type(row[f'Comment.{i}']) is str and row[f'Comment.{i}'] != '':
                url = f"https://{domain}/api/v1/workspaces/{workspace_slug}/projects/{project_id}/issues/{issue_id_table[row['Issue id']]}/comments/"
                payload = build_comment_payload(row[f'Comment.{i}'])
                #print(payload)
                response = send_request("POST", url, headers, payload)

def add_issue_key_comments(domain ,workspace_slug, project_id, df, issue_id_table, commentator_id):
    for _, row in df.sort_values(by=["Issue id"]).iterrows():
        url = f"https://{domain}/api/v1/workspaces/{workspace_slug}/projects/{project_id}/issues/{issue_id_table[row['Issue id']]}/comments/"
        payload = build_comment_payload(f"__;{commentator_id};JIRA Issue key : {row['Issue key']}")
        print(payload)
        response = send_request("POST", url, headers, payload)


def associate_issues_to_cycles(domain, workspace_slug, project_id, df,issue_id_table, cycle_table):
    # associate issues and cycles.
    for _, row in df.sort_values(by=["Issue id"]).iterrows():
        if row['Last Sprint'] != '':
            row_sprint = cycle_table[row['Last Sprint']]
            url = f"https://{domain}/api/v1/workspaces/{workspace_slug}/projects/{project_id}/cycles/{row_sprint}/cycle-issues/"
            payload = {
                "issues" : [issue_id_table[row['Issue id']]]
            }
            response = send_request("POST", url,headers, payload)
            print(response)

def manage_parent_relationships(domain, workspace_slug, project_id, df, issue_id_table):
    # manage parent relationships. Parent may not exist in the df, so check first
    # expect df_issues as input
    if 'Parent' not in df.columns:
        return None
    for _, row in df.sort_values(by=["Issue id"]).iterrows():
        if type(row['Parent']) is str and row['Parent'] != '':
            try:
                url = f"https://{domain}/api/v1/workspaces/{workspace_slug}/projects/{project_id}/issues/{issue_id_table[row['Issue id']]}/"
                payload = {"parent": issue_id_table[row['Parent']]}
                response = send_request("PATCH", url,headers, payload)
            except KeyError:
                #KeyError may occur if this function is used with df_issues and the parent is an EPIC => ignore in that case, since Epic are managed via modules already
                pass


def manage_parent_and_module_relationships(domain, workspace_slug, project_id, df, issue_id_table, module_table):
    # manage parent and module relationships. Expecting df_issues in input
    if 'Parent' not in df.columns:
        return None
    for _, row in df.sort_values(by=["Issue id"]).iterrows():
        if type(row['Parent']) is str and row['Parent'] != '':
            if row['Parent'] in list(module_table.keys()):
                url = f"https://{domain}/api/v1/workspaces/{workspace_slug}/projects/{project_id}/modules/{module_table[row['Parent']]}/module-issues/"
                payload = {
                    "issues" : [issue_id_table[row['Issue id']]]
                    }
                response = send_request("POST", url,headers, payload)
            else:
                url = f"https://{domain}/api/v1/workspaces/{workspace_slug}/projects/{project_id}/issues/{issue_id_table[row['Issue id']]}/"
                payload = {"parent": issue_id_table[row['Parent']]}
                response = send_request("PATCH", url,headers, payload)



def associate_issue_to_modules(domain, workspace_slug, project_id, df, issue_id_table, module_table):
    # Custom field (Epic Link) may not exist in the df, so check first
    if 'Custom field (Epic Link)' not in df.columns:
        return None
    for _, row in df.sort_values(by=["Issue id"]).iterrows():
        if row['Custom field (Epic Link)'] != '':
            url = f"https://{domain}/api/v1/workspaces/{workspace_slug}/projects/{project_id}/modules/{module_table[row['Custom field (Epic Link)']]}/module-issues/"
            payload = {
                "issues" : [issue_id_table[row['Issue id']]]
            }
            response = send_request("POST", url,headers, payload)

In [ ]:
# manage attachments

def get_file_id(file_name, drive_service):
    """
    Retrieves the file ID of an existing file on Google Drive by its name.

    Args:
        file_name: The name of the file to search for.
        drive_service: The authenticated Google Drive API service object.

    Returns:
        The file ID of the file if found, or None if the file does not exist.
    """
    try:
        # Search for the file by name in Google Drive
        results = drive_service.files().list(
            q=f"name='{file_name}' and trashed=false",
            spaces='drive',
            fields='files(id, name)',
            pageSize=10,
            includeItemsFromAllDrives=True,  # Search across all drives
            supportsAllDrives=True  # Enable support for shared drives
        ).execute()

        # Extract the files list from the response
        files = results.get('files', [])

        # Check if any files match the name
        if not files:
            print(f"No file found with the name '{file_name}'.")
            return None

        # If there are multiple matches, return the first one
        file_id = files[0].get('id')
        print(f"Found file: {files[0].get('name')} (ID: {file_id})")
        return file_id

    except Exception as e:
        print(f"Error retrieving file ID: {e}")
        return None


def upload_file_to_gdrive(file_url, filename):
    """Uploads a file from a URL to Google Drive and returns the shareable link.

    Args:
        file_url: The URL of the file to download.
        filename: The desired filename in Google Drive.

    Returns:
        The shareable link of the uploaded file, or None if upload fails.
    """

    try:
        response = requests.get(file_url, stream=True, cookies = cookies_dict)
        response.raise_for_status()  # Raise an exception for bad status codes

        file_content = BytesIO(response.content)

        file_content.seek(0)

        with open(filename, 'wb') as f:
            f.write(file_content.read())

        # Authenticate with Google Drive API

        #file_metadata = {
        #    'name': filename,
        #    'parents': [gdrive_folder_path.split('/')[-1]]  # Assuming the last part of the path is the folder ID
        #}
        #media = MediaIoBaseUpload(file_content, mimetype='application/octet-stream', resumable=True)

        #file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
        #file_id = file.get('id')

    except requests.exceptions.RequestException as e:
        print(f"Error downloading file from URL: {e}")
        return None

def update_permission_get_link(filename):
    try:
        drive_service = build('drive', 'v3')
        # Permissions
        permission = {
            'type': 'anyone',
            'role': 'reader'
        }
        file_id = get_file_id(filename, drive_service)
        drive_service.permissions().create(fileId=file_id, body=permission, supportsAllDrives=True).execute()

        # Get shareable link
        shareable_link = drive_service.files().get(fileId=file_id, fields='webViewLink', supportsAllDrives=True).execute().get('webViewLink')
        return shareable_link


    except Exception as e:
        print(f"Error uploading to Google Drive: {e}")
        return None

# function to create new attachments columns in df, new Gdrive links and update it with new GDrive links

def push_attachments_gdrive_update_df(df):
    count = 0
    if 'Attachment' not in df.columns:
        return df
    for columns in df.columns:
        if columns.startswith('Attachment'):
            df['GDrive.' + str(columns)] = ""
            count += 1
    #print(df.columns.tolist())
    # first create all files in GDrive, then update all permissions and get links
    # otherwise, we would neet to wait several seconds between file creation and link retrieval
    start_time = time.time()
    for _, row in df.sort_values(by=["Issue id"]).iterrows():
        if type(row['Attachment']) is str and row['Attachment'] != '':
            filename = row['Attachment'].split(';')[2]
            filelink = row['Attachment'].split(';')[3]
            upload_file_to_gdrive(filelink, filename)
        for i in range(1, count):
            if type(row['Attachment.' + str(i)]) is str and row['Attachment.' + str(i)] != '':
                filename = row['Attachment.' + str(i)].split(';')[2]
                filelink = row['Attachment.' + str(i)].split(';')[3]
                upload_file_to_gdrive(filelink, filename)
    end_time = time.time()
    if end_time - start_time < 15:
        time.sleep(15-(end_time - start_time))
    for index, row in df.sort_values(by=["Issue id"]).iterrows():
        if type(row['Attachment']) is str and row['Attachment'] != '':
            filename = row['Attachment'].split(';')[2]
            gdrive_link = update_permission_get_link(filename)
            df.at[index, 'GDrive.Attachment'] = gdrive_link
        for i in range(1, count):
            if type(row['Attachment.' + str(i)]) is str and row['Attachment.' + str(i)] != '':
                filename = row['Attachment.' + str(i)].split(';')[2]
                gdrive_link = update_permission_get_link(filename)
                df.at[index, 'GDrive.Attachment.' + str(i)] = gdrive_link
    return df


In [ ]:
def create_comments_attachments(domain, workspace_slug, project_id, df):
    if 'GDrive.Attachment' not in df.columns:
        return None
    count = 0
    for columns in df.columns:
        if columns.startswith('GDrive'):
            count += 1
    for _, row in df.sort_values(by=["Issue id"]).iterrows():
        if type(row['GDrive.Attachment']) is str and row['GDrive.Attachment'] != '':
            url = f"https://{domain}/api/v1/workspaces/{workspace_slug}/projects/{project_id}/issues/{issue_id_table[row['Issue id']]}/comments/"
            file_name = row['Attachment'].split(';')[2]
            file_link = row['GDrive.Attachment']
            payload = {
                "comment_html": f"<p><a target=\"_blank\" rel=\"noopener noreferrer nofollow\" class=\"text-custom-primary-300 underline underline-offset-[3px] hover:text-custom-primary-500 transition-colors cursor-pointer\" href=\"{file_link}\">{file_name}</a></p>"
                }
            response = send_request("POST", url, headers, payload)
        for i in range(1, count):
            if type(row['GDrive.Attachment.' + str(i)]) is str and row['GDrive.Attachment.' + str(i)] != '':
                url = f"https://{domain}/api/v1/workspaces/{workspace_slug}/projects/{project_id}/issues/{issue_id_table[row['Issue id']]}/comments/"
                file_name = row['Attachment.' + str(i)].split(';')[2]
                file_link = row['GDrive.Attachment.' + str(i)]
                payload = {
                    "comment_html": f"<p><a target=\"_blank\" rel=\"noopener noreferrer nofollow\" class=\"text-custom-primary-300 underline underline-offset-[3px] hover:text-custom-primary-500 transition-colors cursor-pointer\" href=\"{file_link}\">{file_name}</a></p>"
                    }
                response = send_request("POST", url, headers, payload)


# Export one project from Jira

In [ ]:
# Make sure there is no future sprint with issues in it : they will be ignored

# get json of sprints using https://CompanyName.atlassian.net/rest/agile/1.0/board/xx/sprint
# board nb (xx) is visible in all urls of a given project
# Either copy and paste json content here :

#sprint_json = json.loads("""{ your json here }""")

# or, if you have already added cookies to connect to atlassian for the attachments
# set url :
url = "https://<your cpy domain name in jira>.atlassian.net/rest/agile/1.0/board/2/sprint"

#requests.request(method, url, headers=headers, json=payload)
response = requests.get(url, cookies = cookies_dict)
sprint_json = json.loads(response.text)


# creates simplified sprint table
# excludes 'future' sprint, assuming there is no issues in there
sprint_table = []
for sprints_count in range(sprint_json['total']):
    if sprint_json['values'][sprints_count]['state'] != 'future':
        sprint_table.append({'name': sprint_json['values'][sprints_count]['name'], 'startDate' : datetime.datetime.fromisoformat(sprint_json['values'][sprints_count]['startDate'].rstrip('Z')), 'endDate' : datetime.datetime.fromisoformat(sprint_json['values'][sprints_count]['endDate'].rstrip('Z'))})

print(sprint_table)

In [ ]:
# import project csv
# use the export function in the issue view in Jira : chose export excel csv (all fields)
# change the filename accordingly

filename ='/content/Jira Export Excel CSV (all fields) timestamp.csv'

types = defaultdict(lambda: str)
with open(filename, 'r') as f:
    df = pd.read_csv(f,dtype=types, keep_default_na=False)

df.head()

In [ ]:
# This gives you the list of members associated with your project in Jira
project_user_list = set(df['Assignee'].tolist())
print(project_user_list)

In [ ]:
# Preparing the project state dict, will be updated with actual state id, depending on your
# state correspondance table defined above
project_jira_state_list = set(df['Status'].tolist())
project_jira_state_dict = {}
for item in project_jira_state_list:
    project_jira_state_dict[item] = 'tbd'
print(project_jira_state_dict)

In [ ]:
#This cell tells you if there are attachments to manage in your Jira project (empty list means no attachment)

[column_names for column_names in df.columns.tolist() if column_names.startswith('Attachment')]

# Import / create in Plane : execution

In [ ]:
# FIRST create project in Plane UI and ADD NECESSARY MEMBERS (C.F. list above)
# member addition cannot be made via API
# THEN enter project id => can be found in project view url, format looks like example below
project_id = "d7b51669-a4a4-4d45-987c-1f93458d3939"

In [ ]:
# Get project details (this will allow you to check your project id is ok, check number of members)

url = f"https://{domain}/api/v1/workspaces/{workspace_slug}/projects/{project_id}/"

response = requests.request("GET", url, headers=headers)
response = json.loads(response.text)
print(response)

In [ ]:
#Prepare data : run as is

#assumes sprint_table & df are created

project_jira_state_dict = complete_state_dict(domain, workspace_slug, project_id)
label_bug = create_or_get_label_bug(domain, workspace_slug, project_id)

cycle_table = {}
issue_id_table = {}
module_table = {}

df = add_lastsprint_df(df)

# define what you consider issues in Plane. By default Bug will be issues flagged with a particular Label
# Story & Task will be delt without distinction
# Sub task will become subtasks thanks to parenting
# epic will be become modules
df_issues = df[df['Issue Type'].isin(['Story', 'Task', 'Sub-task', 'Bug', 'Subtask'])]
df_epic = df[df['Issue Type'] == 'Epic']

In [ ]:
# Create modules, if any : run as is
module_table = create_module_v2(domain, workspace_slug, project_id, df_epic, module_table)

In [ ]:
# Create issues : run as is

# first : manage the case where there are no sprints in the project
if 'Sprint' not in df.columns:
    issue_id_table = create_issues(domain, workspace_slug, project_id, df_issues, project_jira_state_dict, label_bug, issue_id_table)

else:
    # create issues without sprints first :
    issue_id_table = create_issues(domain, workspace_slug, project_id, df_issues[df_issues['Last Sprint']==''], project_jira_state_dict, label_bug, issue_id_table)

    # create issues in sprint, from sprint chronological order
    # first : create one cycle without end date, then create issues, associate them, then add closing dates
    for sprint in sprint_table:
        cycle_table = create_cycle_no_end_date(domain, workspace_slug, project_id, cycle_table, [sprint])
        issue_id_table = create_issues(domain, workspace_slug, project_id, df_issues[df_issues['Last Sprint']==sprint['name']], project_jira_state_dict, label_bug, issue_id_table)
        associate_issues_to_cycles(domain, workspace_slug, project_id, df_issues[df_issues['Last Sprint']==sprint['name']], issue_id_table, cycle_table)
        update_one_cycle_end_date(domain, workspace_slug, project_id, cycle_table, [sprint])

In [ ]:
# Add comments : run as is

# Add all comments from Jira
add_comments(domain, workspace_slug, project_id, df_issues, issue_id_table)

# add previous (Jira) issue key as additional comment. Provide a valid commentator id as input
add_issue_key_comments(domain ,workspace_slug, project_id, df_issues, issue_id_table, commentator_id = '5e1d869d010b260ca879be6f')

In [ ]:
# manage all parent relationships (sub tasks, and epic) : run as is
manage_parent_and_module_relationships(domain, workspace_slug, project_id, df_issues, issue_id_table, module_table)

In [ ]:
# create attachments from jira in GDrive : run as is
df= push_attachments_gdrive_update_df(df)

In [ ]:
# create attachment links as comments in Plane  : run as is
create_comments_attachments(domain, workspace_slug, project_id, df)